- Train a simple baeline with Transormers & Datasets

In [1]:
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
import sys
sys.path.insert(1, '/tf/local/baneshi/PersianQA/')

In [2]:
# model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 2048 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 4e-5
epoch = 20

In [3]:
datasets = load_dataset("SajjadAyoubi/persian_qa")
datasets['train'][0]

Reusing dataset persian_qa (/home/jovyan/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225)


  0%|          | 0/2 [00:00<?, ?it/s]

{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [5]:
# datasets = load_dataset("SajjadAyoubi/persian_qa")

from src.read_ds import read_qa
train_dataset = read_qa("../dataset/pqa_train.json")
test_dataset = read_qa("../dataset/pqa_test.json")
dataset = {}
dataset['train'] = train_dataset
dataset['test'] = test_dataset
dataset = DatasetDict(dataset)
dataset['train'][0]


In [4]:
model_checkpoint = "../models/roberta-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [6]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_ds = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225/cache-2612c32d9af815ca.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at ../models/roberta-fa-zwnj-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at ../models/roberta-fa-zwnj-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probab

In [19]:
# from datasets import load_dataset,DatasetDict
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# from transformers import TrainingArguments, Trainer, AdamW
# import torch


# from torch.utils.data import DataLoader
# from transformers import AdamW

# device = torch.device('cuda')

# model.to(device)
# model.train()

# train_loader = DataLoader(tokenized_ds['train'], batch_size=8, shuffle=True, drop_last=True)
# optim = AdamW(model.parameters(), lr=5e-5)
# print(train_loader)
# for epoch in range(2):
#     for batch in train_loader:
#         print(batch)
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         start_positions = batch['start_positions'].to(device)
#         end_positions = batch['end_positions'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()

In [8]:
args = TrainingArguments(
    output_dir=f"result"+"/"+model_checkpoint.split("/")[-1]+"__PQA_29",
    evaluation_strategy = "epoch",
    save_steps = 2000,
    logging_strategy = "epoch",
    logging_steps = 2000,
    logging_dir = f"result"+"/"+model_checkpoint.split("/")[-1]+"__PQA_29"+"/logs",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    hub_strategy = "checkpoint",
    save_total_limit = 12,
    hub_model_id = "1",
    weight_decay=0.01) 


In [8]:
args = TrainingArguments(
    output_dir=f"result"+"/"+model_checkpoint.split("/")[-1]+"__PQA_29",
    evaluation_strategy = "steps",
    save_steps = 2000,
    logging_strategy = "steps",
    logging_steps = 2000,
    logging_dir = f"result"+"/"+model_checkpoint.split("/")[-1]+"__PQA_29"+"/logs",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    hub_strategy = "checkpoint",
    save_total_limit = 12,
    hub_model_id = "1",
    weight_decay=0.01) 

In [9]:
import transformers
grouped_params = model.parameters()
optimizer=transformers.AdamW(grouped_params,lr=lr)

# optimizer = transformers.Adafactor(
#     model.parameters(),
#     lr=4e-6,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False,
# )


scheduler=transformers.get_cosine_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=1000, num_training_steps=10000)
optimizers = optimizer, scheduler

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
# start training
trainer.train()
# trainer.train(resume_from_checkpoint="result/distil-bigbird-fa-zwnj__PQA_4/checkpoint-2000")

***** Running training *****
  Num examples = 9008
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 22520


Epoch,Training Loss,Validation Loss


RuntimeError: The expanded size of the tensor (523) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [8, 523].  Tensor sizes: [1, 514]